# Portfolio Optimisation

A 1/N portfolio with equal weights is amongst portfolio formation strategies like naive diversification, where we select assets because they differ in kind, industry or something else. With this strategy we don't look closely at return or risk, but rather depend on the diversity of our assets.

Other formation strategies do analyse return and risk, and attempt to optimise weights based on some criteria. Portfolio optimisation involves determining the best allocation of our assets to achieve some specific investment objective. Let's optimise our portfolio weights and see what it can do!


In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

plt.style.use("ggplot")

RFR = 0.0441
TRADING_DAYS = 252

Above we've got our imports, settings and a constant, and below we've got our data frame being created and checked. We're also selecting the 5 stocks of our portfolio again.

In [24]:
df = pd.read_csv("data/top_six_2020_2025.csv")
df.DlyCalDt = pd.to_datetime(df.DlyCalDt, dayfirst=True)

df_pivot = df.pivot(index="DlyCalDt", columns="Ticker", values="DlyClose")
df_pivot = df_pivot[df_pivot.columns[:5]]

returns = df_pivot.pct_change().dropna()

df_pivot

## Lambdas and Comprehensions

Before we get into optimisation, there are two constructs you'll need to know within Python. These are used all over the language (and in other programming languages as well).

First up we've got `lambda`, or anonymous, functions. If you're in need of a function but don't want to spend time writing one, a lambda function is for you.

In [25]:
# We can assign them to variables
hello = lambda x: print("Hello", x)
hello("World")

# but we can pass them as arguments if we want
returns.apply(lambda x: x * 100) # percentage returns !

Next is **comprehensions**. If lambdas are one-line functions then comprehensions are one-line for loops.


In [26]:
[print("Hello World") for x in range(3)]

# Let's square some numbers
nums = [1, 2, 3, 4]
[num ** 2 for num in nums]

## Optimise with SciPy

SciPy is a computational library for Python that goes beyond NumPy by adding lots of useful functions for scientific fields. One of the useful functions it has is called `minimize()`, which is used for solving optimisation problems across STEM. This function attempts to minimise some value given a set of input values. In our case, we will be trying to minimise the negative Sharpe Ratio (or maximising the Sharpe Ratio if you prefer!) by finding the optimal portfolio weights.

Recall that Sharpe Ratio is a measure of risk-adjusted return. The higher the value, the higher return we get for the risk we put in.

To work with `minimize()` we need a function. So let's define a function to calculate the negative Sharpe Ratio. It should accept weights as an argument, since this is the parameter we'll be optimising to get the Sharpe Ratio higher.

In [27]:
def negative_sharpe(weights):
    portfolio_returns = np.dot(returns, weights)
    annualised_returns = portfolio_returns.mean() * TRADING_DAYS
    annualised_volatility = portfolio_returns.std() * np.sqrt(TRADING_DAYS)

    sharpe_ratio = (annualised_returns - RFR) / annualised_volatility

    return -sharpe_ratio

Then we need to define the **bounds** and **constraints**, and the *intial weights*.

The bounds restrict the values that `minimize()` will attempt to use for individual inputs (weights in our case). We'll need five weights, and weights should be greater than or equal to 0 and less than or equal to 1.

The constraints restrict the possible solutions that `minimize()` can return. For example, in our solution, all the weights must sum to 1.0, such that our portfolio is complete. 

For the initial weights we'll start with a 1/N portfolio.

In [28]:
# Constraints
cons = {"type": "eq", "fun": lambda x: sum(x) - 1}

# Bounds
bounds = tuple((0, 1) for x in range(5))

# Weights
init_weights = np.ones(5) / 5

optimal = minimize(negative_sharpe, init_weights, bounds=bounds, constraints=cons)
optimal_weights = optimal.x
optimal_weights

## Backtesting Optimal Weights

We now have the weights that maximise the Sharpe Ratio. Let's use these weights to see how our portfolio would have performed historically.

### Exercise: Watch your back

Use the `optimal_weights` to calculate new portfolio returns, and daily volatility. How do these annualised values compare to the 1/N portfolio we started with?

In [29]:
## YOUR CODE GOES HERE

Next, plot the optimal weight portfolio cumulative returns and the initial equal weight cumulative returns together on a plot to see how we did compared to the benchmark.

In [ ]:
## YOUR CODE GOES HERE